In [ ]:
%load_ext nb_black
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import numpy as np
import pandas as pd

from torch.utils.data import Dataset, DataLoader

import sys

sys.path.append("../code/")
import glob
from data.dataset import InMemoryTrainDataset

from matplotlib import pyplot as plt

In [ ]:
TRAIN_SIZE = 256
REDUCE_FACTOR = 4

df_rle = pd.read_csv(f"../input/train_{REDUCE_FACTOR}.csv")

In [ ]:
train_img_names = df_rle.id.unique()

In [ ]:
from data.transforms import HE_preprocess

viz_ds = InMemoryTrainDataset(
    train_img_names[:5],
    df_rle,
    train_tile_size=TRAIN_SIZE,
    reduce_factor=REDUCE_FACTOR,
    train_transfo=HE_preprocess(augment=True, visualize=True, size=TRAIN_SIZE),
    valid_transfo=HE_preprocess(augment=False, visualize=True, size=TRAIN_SIZE),
    train_path=f"../input/train_{REDUCE_FACTOR}/",
    iter_per_epoch=100,
    sampling_mode="centered",  # TODO 'convhull',
    use_external=0.2,  # None, #0.2,
    on_spot_sampling=1,  # 0.99
    oof_folder="../logs/2021-04-05/4/",
)

In [ ]:
img, mask, oof_pred = viz_ds[10]

plt.figure(figsize=(15, 5))
plt.subplot(1, 2, 1)
plt.imshow(mask)
plt.subplot(1, 2, 2)
plt.imshow(oof_pred)

plt.show()

In [ ]:
# viz_ds.iter_per_epoch = 1000
# viz_ds.on_spot_sampling = 0.2
# viz_ds.sampling_mode='centered' # 'visible'
# viz_ds.train_tile_size=512
# viz_ds.train_transfo = HE_preprocess(augment=True, visualize=True, size=512)
# viz_ds.valid_transfo = HE_preprocess(augment=False, visualize=True, size=512)
# viz_ds.train(True)

In [ ]:
from utils.plots import plot_contours

viz_dl = DataLoader(viz_ds, batch_size=64, shuffle=True)
imgs, masks, oof = next(iter(viz_dl))

plt.figure(figsize=(16, 16))
for i, (img, mask) in enumerate(zip(imgs, masks)):
    img = img.permute(1, 2, 0).numpy()  # .astype(np.uint8)
    plt.subplot(8, 8, i + 1)

    plot_contours(img.copy(), mask.numpy(), w=2)
    plt.axis("off")
    plt.subplots_adjust(wspace=None, hspace=None)

In [ ]:
viz_ds.train(False)

In [ ]:
from utils.plots import plot_contours

viz_dl = DataLoader(viz_ds, batch_size=64, shuffle=True)
imgs, masks, _ = next(iter(viz_dl))

plt.figure(figsize=(16, 16))
for i, (img, mask) in enumerate(zip(imgs, masks)):
    img = img.permute(1, 2, 0).numpy()  # .astype(np.uint8)
    plt.subplot(8, 8, i + 1)

    plot_contours(img.copy(), mask.numpy(), w=2)
    plt.axis("off")
    plt.subplots_adjust(wspace=None, hspace=None)
print(img.shape)